In [22]:
import pandas as pd
import numpy as np
import os
import glob
import sys

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
WIN = 'D:/'
MAC = '/Volumes/Transcend2/17_20_PhD/19_20_PhD_Metabolomics/'

osp = MAC

## Load in limma results

Data was obtained in Step 1/6.PeakSetFiltering

In [4]:
output_dir = osp+'/pymz/mzmine/'
limma_filename = 'limma_independent_toptable_features.csv'


## Run mummichog

In [5]:
keyword = 'mm211020'
os.system('mummichog -f'+ output_dir + limma_filename +
          ' -m positive -u 3 -k MummichogResults -o '+ keyword+' -c 0.05')

0

## Annotate data

In [6]:
for f in os.listdir('MummichogResults'):
    if keyword in f:
        mummichog_folder = f

In [7]:
compounds = pd.read_csv('MummichogResults/'+mummichog_folder+'/tables/ListOfEmpiricalCompounds.tsv', sep='\t')
featureoutput = pd.read_csv('MummichogResults/'+mummichog_folder+'/tables/userInputData.txt', sep='\t')
pathway = pd.read_csv('MummichogResults/'+mummichog_folder+'/tables/mcg_pathwayanalysis_'+keyword+'.tsv', sep='\t')

In [8]:
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    matching_ids[key] = value

In [9]:
compounds_dict = {}
for _,row in compounds.iterrows():
    ids = row['massfeature_rows']
    ions = row['str_row_ion']
    compoundlist = row['compound_names']
    kegg_id = row['compounds']
    
    
    ids_list = ids.split(';')
    ions_list = ions.split(';')
    for i in range(len(ids_list)):
        compounds_dict[matching_ids[ids_list[i]]] = []
        compounds_dict[matching_ids[ids_list[i]]].append(ions_list[i].split('_')[1])
        compounds_dict[matching_ids[ids_list[i]]].append(compoundlist)
        compounds_dict[matching_ids[ids_list[i]]].append(kegg_id)
        

In [10]:
ecomp_dict = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    massfeature_rows = row['massfeature_rows'].split(';')
    for mass in massfeature_rows:
        if mass in ecomp_dict:
            ecomp_dict[mass].append(eid)
        else:
            ecomp_dict[mass] = []
            ecomp_dict[mass].append(eid)
            
cmpd_names = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    name = row['compound_names']
    cmpd_names[eid] = name
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    
    matching_ids[value] = key
for n in matching_ids:
    if matching_ids[n] in ecomp_dict:
        matching_ids[n] = ecomp_dict[matching_ids[n]]

id_names = {}
for i in matching_ids:
    if type(matching_ids[i]) is list:
        for f in matching_ids[i]:
            if f in cmpd_names:
                if i in id_names:
                    id_names[i].append(cmpd_names[f])
                else:
                    id_names[i] = []
                    id_names[i].append(cmpd_names[f])
matching_ids2 = {}
for n in matching_ids:
    
    if type(matching_ids[n]) is list:
        lista = matching_ids[n]
        for p in lista:
            if p in matching_ids2:
                matching_ids2[p].append(n)
            else:
                matching_ids2[p] = []
                matching_ids2[p].append(n)
pathway_dict = {}
for _,row in pathway.iterrows():
    
    pname = row['pathway']
    idlist =row['overlap_EmpiricalCompounds (id)']
    
    if type(idlist) != float:
        idlist = idlist.split(',')
        for p in idlist:
            
            if pname in pathway_dict:
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
            else:
                pathway_dict[pname] = []
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
pathway_dict2 = {}
for p in pathway_dict:
    
    for n in pathway_dict[p]:
        for i in n:
            if i in pathway_dict2:
                if p not in  pathway_dict2[i]:
                    pathway_dict2[i].append(p)
            else:
                pathway_dict2[i] = []
                pathway_dict2[i].append(p)
                

In [11]:
data = pd.read_csv(output_dir+limma_filename, sep='\t')

In [12]:
data.index = data['custom_id']

In [23]:
sys.path.append('..')
import peakinfo
with open(osp+'pymz/mzmine/peak_picked_files/peakinfolist.dict', 'rb') as file:
    peakinfolist = pickle.load(file)

In [25]:
for row_id,row in data.iterrows():
    peak = peakinfo.get_peak_by_cid(peakinfolist, row_id)
    if row_id in id_names:
        peak.add_mm_annotation([str(name).lower() for name in id_names[row_id]])
    if row_id in pathway_dict2:
        peak.add_mm_pathway(pathway_dict2[row_id])
    if row_id in compounds_dict:
        peak.add_mm_kegg_id(compounds_dict[row_id][2])   

In [26]:
with open(osp+'pymz/mzmine/peak_picked_files/peakinfolist.dict', 'wb') as handle:
    pickle.dump(peakinfolist, handle)

## Matching with standards ToxID annotation

In [15]:
import pickle
with open(osp+'Metabolomics_datasets/standards_list.dict', 'rb') as file:
    standards_dict = pickle.load(file)

In [17]:
peakset = {}
for _, row in data.iterrows():
    cid = row['custom_id']
    mz = row['row m/z']
    rt = row['row retention time']
    pval = row['adj-p']
    tval = row['t-val']
    mm_name = row['mm_compound_names']
    mm_path = row['mm_pathway']
    mm_kegg = row['mm_kegg_id']
    std = row['std_match']
    
    peakset[cid] = (mz, rt*60, pval, tval, mm_name, mm_path, mm_kegg, std)

In [18]:
with open(osp+'pymz/mzmine/peak_picked_files/filtered_peaksets.dict', 'wb') as handle:
    pickle.dump(peakset, handle)

## Check accuracy of mummichog annotation

In [19]:
notannotated = 0
annotated = 0
notannotateds = 0
annotateds = 0
match = 0
matchs =0
for i,row in data.iterrows():
    cmp = str(row['mm_compound_names']).lower()
    std = str(row['std_match']).lower()
    
    if cmp in ['nan']:
        notannotated +=1
        if not std in ['nan']:
            matchs +=1
           
    else:
        annotated +=1
        if not std in ['nan']:
            if std in cmp:
                match +=1
            else:
                print(row)
    
    if std in ['nan']:
        notannotateds +=1
    else:
        annotateds +=1
        
percentage_exact_matches = match*100/annotateds
percentage_not_annotated = matchs*100/annotateds
percentage_not_matched = (annotateds-match-matchs)*100/annotateds

print("Total number of annotated compounds using mummichog: {}".format(annotated))
print("Total number of annotated compounds using ToxID standards: {}".format(annotateds))
print("Total number of annotation matches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(match, percentage_exact_matches))
print("Total number of annotation mismatches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(annotateds-match-matchs, percentage_not_matched))
print("Total number of metabolites annotated with ToxID, but not with mummichog: {0}, i.e. {1:.2f}%".format(matchs, percentage_not_annotated))

row m/z                                               148.073
row retention time                                    11.9154
adj-p                                             2.36533e-07
t-val                                                -7.29106
custom_id                                                 633
mm_compound_names                            [(r)-mevalonate]
mm_pathway            [Squalene and cholesterol biosynthesis]
mm_kegg_id                                             C00418
std_match                                   o-acetyl-l-serine
Name: 633, dtype: object
row m/z                                                         136.043
row retention time                                              10.0993
adj-p                                                        0.00013607
t-val                                                          -4.73328
custom_id                                                           459
mm_compound_names                                      [l